## 変分オートエンコーダ(VAE)

- ニューラルネットワークを用いた生成モデルの一種である, 変分オートエンコーダ(Variational Auto Encoder)に関するメモ.
- [Kerasのサンプルコード](https://keras.io/examples/generative/vae/)も実行してみた.

---

### 1. 変分オートエンコーダ(VAE)

> **Point**<br>
> ✅ **変分オートエンコーダ**(VAE)は, ニューラルネットワークを用いた生成モデルの一種. <br>
> ✅ VAEは**符号化器**(エンコーダ)と**復号化器**(デコーダ)とそれらの間の**中間層**からなる.<br>
> ✅ 画像データなどをVAEに教師なし学習を行うことで, データをよく説明する低次元の潜在パラメータ空間を構成でき, データ生成が可能である.

#### 1-1. 構成

VAEは以下の3つの層を備えたニューラルネットワークである.
- **符号化器**(エンコーダ)：入力データを中間層に送り, 低次元の特徴量に圧縮する. 
- **中間層**：入力データの特徴を捉えた潜在的なパラメータ空間.
- **復号化器**(デコーダ)：中間層のデータを元のデータに戻す.

#### 1-2. 生成モデル

VAEのデータの生成過程について述べる. はじめにいくつか記号を導入する：

- 入力データ $\underline{x} \in {\mathbb R}^d$
- 中間層の特徴量ベクトル $\underline{z} \in {\mathbb R}^q$
- 符号化器 $F(\cdot; \theta)$. 入力データを潜在パラメータに変換する. パラメータ$\theta$を持つ(NNの重み/バイアス)
- 復号化器 $G(\cdot; \phi)$. 潜在パラメータを元のデータに復元する. パラメータ$\phi$を持つ(NNの重み/バイアス)

<!-- 標本$X = \{\underline{x}^{(1)}, \ldots, \underline{x}^{(N)}\}$プロセスによって生成されると仮定する： -->

**データの生成過程**

1. 潜在パラメータ$\underline{z}$の事前分布は多次元標準正規分布${\mathcal N}(0, I_q)$に従う.
2. 出力$\underline{x}$は潜在パラメータ$\underline{z}$に依存して次のように確率的に生成される$$\underline{x} \vert \underline{z} \sim p_\theta(\underline{x} \vert \underline{z})$$
    ※ $p_\theta$は入力データの性質および計算コストを考慮して適当にモデリングを行う必要がある. VAEではこのモデリングにNNを用いる. 

入力$\underline{x}$に対して, 潜在パラメータの事後分布$p_\theta(\underline{z}\vert \underline{x})$を知ることが目標である. しかし, この分布を直接求めることは困難なため**近似推論**を利用する
- 試験分布$q_\phi(\underline{z} \vert \underline{x})$により$p_\theta(\underline{z} \vert \underline{x})$を近似(ただし$\phi$はパラメータである.)
- どの程度よく近似されているかの尺度には次の**変分下界**(variational lower bound)(または, **エビデンス下限**(ELBO: evidence lower bound))を用いる：
    $$\mathcal{L}(\theta, \phi, \underline{x}) := \log{p_\theta(\underline{x})} - {\rm KL}(q_\phi(\underline{z}\vert\underline{x})||p_\theta(\underline{z}\vert \underline{x}))$$
    ただし, ${\rm KL}$はカルバック・ライブラー情報量と呼ばれる確率分布同士の近さの尺度であり, 確率密度関数$p, q$に対し次で定義される：
    $${\rm KL}(p || q) :=  \int p(z) \log{\frac{p(z)}{q(z)}}dz.$$

近似推論の教えるところは以下の通りである

> **近似推論**<br>
> 変分下限を最大化する$q_\phi(\underline{z}\vert\underline{x})$は$p_\theta(\underline{z}\vert \underline{x})$をよく近似する.

**符号化器**

VAEでは試験分布$q_\phi(\underline{z} \vert \underline{x})$として, 次のような多次元正規分布を仮定する：

$$
q_\phi(\underline{z} \vert \underline{x}) = {\mathcal N}(\underline{\mu}(\underline{x}), \Sigma(\underline{x}))
$$
ただしパラメータ
$$
\begin{align*}
\underline{\mu}(\underline{x}) &= (\mu_1(\underline{x}), \ldots, \mu_q(\underline{x}))\\
\Sigma(\underline{x}) &= {\rm diag}(\sigma^2_1(\underline{x}), \ldots, \sigma^2_q(\underline{x}))
\end{align*}
$$
は, 入力データから符号化器(適当なNNモデル)を用いて計算する. 

#### 1-3. 変分下限の評価

変分下限を以下のように式変形：

$$
\begin{align*}
\mathcal{L}(\theta, \phi, \underline{x}) &= \log{p_\theta(\underline{x})} - {\rm KL}(q_\phi(\underline{z}\vert\underline{x})||p_\theta(\underline{z}\vert \underline{x}))\\
&= \log{p_\theta(\underline{x})} - \int q_\phi(\underline{z}\vert\underline{x}) \log \frac{q_\phi(\underline{z}\vert\underline{x})}{p_\theta(\underline{z}\vert \underline{x})} dz\\
&= \mathbb{E}_{q_\phi(\underline{z}\vert\underline{x})}[\log{p_\theta(\underline{x})}] - \mathbb{E}_{q_\phi(\underline{z}\vert\underline{x})}\Bigl[\log \frac{q_\phi(\underline{z}\vert\underline{x})}{p_\theta(\underline{z}\vert \underline{x})}\Bigr] \\
&= \mathbb{E}_{q_\phi(\underline{z}\vert\underline{x})}\Bigl[\log{p_\theta(\underline{x})} - \log \frac{q_\phi(\underline{z}\vert\underline{x})}{p_\theta(\underline{z}\vert \underline{x})}\Bigr]\\
&= \mathbb{E}_{q_\phi(\underline{z}\vert\underline{x})}\Bigl[- \log \frac{q_\phi(\underline{z}\vert\underline{x})}{p_\theta(\underline{z}\vert \underline{x})p_\theta(\underline{x})}\Bigr]\\
&= \mathbb{E}_{q_\phi(\underline{z}\vert\underline{x})}\Bigl[- \log \frac{q_\phi(\underline{z}\vert\underline{x})}{p_\theta(\underline{x}\vert \underline{z})p_\theta(\underline{z})}\Bigr]\\
&= \mathbb{E}_{q_\phi(\underline{z}\vert\underline{x})}[\log{p_\theta(\underline{x}\vert \underline{z})}] - {\rm KL}(q_\phi(\underline{z}\vert\underline{x})||p_\theta(\underline{z}))
\end{align*}
$$

**第1項の評価**

変分下限の第1項は直接評価することが困難なため, モンテカルロ法による近似計算を行う. $\underline{z}_1, \ldots, \underline{z}_M$を分布$q_\phi(\underline{z} \vert \underline{x})$から発生させた乱数として, 
$$
\mathbb{E}_{q_\phi(\underline{z}\vert\underline{x})}[\log{p_\theta(\underline{x}\vert \underline{z})}] \approx \frac{1}{M} \sum_{j = 1}^M \log p_\theta(\underline{x}\vert \underline{z}_m)
$$
と近似する.

**第2項(KLダイバージェンス)の評価**

変分下限の第2項は具体的に書き下すことができる：
$$
\begin{align*}
{\rm KL}(q_\phi(\underline{z}\vert\underline{x})||p_\theta(\underline{z})) &= \mathbb{E}_{q_\phi(\underline{z}\vert\underline{x})}\Bigl[
    \log{\frac{q_\phi(\underline{z}\vert\underline{x})}{p(z)}}
\Bigr]\\
&= \mathbb{E}_{q_\phi(\underline{z}\vert\underline{x})}\Bigl[\log{q_\phi(\underline{z}\vert\underline{x})} \Bigr] + \mathbb{E}_{q_\phi(\underline{z}\vert\underline{x})}\Bigl[- \log{p(z)}\Bigr]\\
&= - \frac{1}{2} (q\log(2\pi e) + \log{|\Sigma(\underline{x}) |}) + \mathbb{E}_{q_\phi(\underline{z}\vert\underline{x})}\Bigl[\frac{q}{2}\log{(2\pi)} + \frac{1}{2} \underline{z}^\top \underline{z}\Bigr] \\
&= - \frac{q}{2} - \frac{1}{2}\log{|\Sigma(\underline{x})|} + \frac{1}{2}||\underline{\mu}(\underline{x})||^2_2 + \frac{1}{2}{\rm tr}(\Sigma(\underline{x}))\\
&= - \frac{1}{2} \sum_{j=1}^q \{1 + \log \sigma_j^2(\underline{x})- \mu_j(\underline{x})^2 - \sigma_j^2(\underline{x})\}
\end{align*}
$$

上記をまとめて次を得る.

> **VAEの目的関数**<br>
> 変分下限の推定量(の-1倍)の推定量
> $$
> - \mathcal{L}(\theta, \phi, \underline{x}) \approx {\rm KL}(q_\phi(\underline{z}\vert\underline{x})||p_\theta(\underline{z})) - \frac{1}{M} \sum_{j = 1}^M \log p_\theta(\underline{x}\vert \underline{z}_m)$$
> に関して最小化するパラメータ$\phi, \theta$が求めるべきパラメータである.

#### 1-4. 再パラメータ化

目的関数$-\mathcal{L}(\theta, \phi, \underline{x})$の最適化は, 確率的勾配降下を用いて行う. 確率的勾配降下では, 最適化パラメータに関する微分$\nabla_{\theta, \phi}(-\mathcal{L}(\theta, \phi, \underline{x}))$を書き下す必要がある. ここで「モンテカルロ法に用いる乱数$\underline{z}_1, \ldots, \underline{z}_M \sim q_\phi(\underline{z} \vert \underline{x})$たちは, $\phi$に依存しているため, $\partial \underline{z}_i/\partial \theta$が微分不可能となってしまう」という問題が生じる.

これに対処するのが**再パラメータ化**(reparametrization trick)というアイデアである.

> **再パラメータ化**(reparametrization trick)<br>
> $\epsilon_1, \ldots, \epsilon_M$を多次元標準正規分布${\mathcal N}(0, I_q)$からの乱数とする. $q_\phi(\underline{z} \vert \underline{x})$からの乱数として
> $$z_i = \underline{\mu}(\underline{x}) + \Sigma^{1/2}(\underline{x})\odot\epsilon_i$$
> を用いることで, $\partial \underline{z}_i/\partial \theta$が計算可能になる(ただし, $\odot$は成分同士の積を表す).

---

### 2. 手書き数字画像生成(MNISTデータセット)

本セクションの内容は, 以下のページのコードサンプルを実行したものです.

[Variational AutoEncoder｜Code Example - Keras](https://keras.io/examples/generative/vae/)

#### 1. setup

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#### 2. モデルの構築

In [2]:
"""
潜在パラメータのサンプリング層

Args:
    [z_mean, z_log_var]: 多次元ガウス分布のハイパーパラメータ
"""

class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [ ]:
"""
encoder層の構築
入力データ型: (28, 28, 1) 28*28のグレースケール画像
出力データ型:  
"""
latent_dim = 2 # 潜在パラメータの次元

encoder_inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
# 抽出された特徴量x(16次元vect)から, 潜在パラメータのハイパーパラメータ(z_mean, z_log_var)を作る
z_mean = layers.Dense(latent_dim, name="z_mean")(x) 
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
# 潜在パラメータをサンプリング
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

In [ ]:
"""
dencoder層の構築
入力データ型: (latent_dim, 1) (潜在パラメータ数)次元ベクトル
出力データ型: (None, 28, 28, 1) -> 元の入力データト同じ次元
"""
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(7 * 7 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((7, 7, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

In [5]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

#### 3. モデルの訓練

In [ ]:
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
mnist_digits = np.concatenate([x_train, x_test], axis=0)
mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(mnist_digits, epochs=30, batch_size=128);

#### 4. 画像生成

In [ ]:
import matplotlib.pyplot as plt


def plot_latent_space(vae, n=30, figsize=15):
    # display a n*n 2D manifold of digits
    digit_size = 28
    scale = 1.0
    figure = np.zeros((digit_size * n, digit_size * n))
    # linearly spaced coordinates corresponding to the 2D plot
    # of digit classes in the latent space
    grid_x = np.linspace(-scale, scale, n)
    grid_y = np.linspace(-scale, scale, n)[::-1]

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[xi, yi]])
            x_decoded = vae.decoder.predict(z_sample)
            digit = x_decoded[0].reshape(digit_size, digit_size)
            figure[
                i * digit_size : (i + 1) * digit_size,
                j * digit_size : (j + 1) * digit_size,
            ] = digit

    plt.figure(figsize=(figsize, figsize))
    start_range = digit_size // 2
    end_range = n * digit_size + start_range
    pixel_range = np.arange(start_range, end_range, digit_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.imshow(figure, cmap="Greys_r")
    plt.show()


plot_latent_space(vae)

In [ ]:
def plot_label_clusters(vae, data, labels):
    # display a 2D plot of the digit classes in the latent space
    z_mean, _, _ = vae.encoder.predict(data)
    plt.figure(figsize=(12, 10))
    plt.scatter(z_mean[:, 0], z_mean[:, 1], c=labels)
    plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.show()


(x_train, y_train), _ = keras.datasets.mnist.load_data()
x_train = np.expand_dims(x_train, -1).astype("float32") / 255

plot_label_clusters(vae, x_train, y_train)

---

### 3. 音楽への応用のための今後の課題

**和音の生成モデルへの応用、および和音の潜在パラメーター空間の可視化**<br>
✅ VAEモデルで和音のデータを学習させ, 和音の特徴量を抽出できるか.<br>
✅ 和音データの収集方法, どの程度用意する必要があるか? 音楽のジャンル?<br>

---

### 4. References

[梅津・西井・上田(2020) 『スパース回帰分析とパターン認識』講談社サイエンティサイエンティフィック ](https://www.kspub.co.jp/book/detail/5186206.html)